# 当代人工智能实验一：文本分类
## ——TF-IDF & Logistic Regression

### 一. 引入必要模块
numpy将用于数据的处理。
CountVectorizer, TfidfVectorizer, TfidfTransformer是在利用TF-IDF方法将文本映射为向量。

In [10]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

### 二. 读取训练与测试数据
每条训练集数据都包含“文本”与给定的“标签”。每条测试集数据都有编号与“文本”。

In [11]:
# 打开并读取训练数据文档
f_train = open('train_data.txt')
train_text = f_train.read()
# print(train_text)
# 观察数据特征，确定数据文档中的每一项均由一个回车分割，故采取切片
train_text = train_text.split("\n")
# 创造两个数组，存储训练数据
# labelList 存储每个数据的标签
# rawList 存储每个数据的文本内容
labelList = []
rawList = []
for i in range(len(train_text)-1):
    train_text[i] = eval(train_text[i])
    labelList.append(train_text[i]["label"])
    rawList.append(train_text[i]["raw"])
labelList = np.array(labelList)
rawList = np.array(rawList)

In [12]:
# 打开并读取测试数据文档
f_test = open('test.txt')
result = f_test.read()
# print(test_text)
# 观察数据特征，确定数据文档中的每一项均由一个回车分割，故采取切片
result = result.split("\n")
# 去除第一行
result.pop(0)
# 测试集的大小
TEST_LENGTH = 2000
result_id = list(range(TEST_LENGTH))
result_text = []
for i in range(TEST_LENGTH):
    comma_index = result[i].find(",")
    if comma_index != -1:
        result_text.append(result[i][comma_index+2:])
    else:
        print("ERROR: COMMA NOT FOUND")
result_text = np.array(result_text)
# result_text

In [13]:
ALL = np.append(rawList, result_text)
# 记录数据的个数
LENGTH_TRAIN = len(rawList)
LENGTH_ALL = len(ALL)
# ALL

### 三. 文本转化为向量
这里使用TF-IDF方法。

In [14]:
# 初始化训练参数
# analyzer为word时，表示以单词为单位计算TF-IDF值
# stop_words为english时，表示去除英语中的停用词，避免缺乏实际意义的计算
# use_idf为True时，表示要计算idf值
# smooth_idf为True时，表示要在文档频率上加一来平滑idf，避免分母为0
# norm为None时，表示输出结果不需要任何标准化或归一化，为l2时，说明输出结果进行了归一化
tv1 = TfidfVectorizer(analyzer="word", stop_words="english", use_idf=True, smooth_idf=True, norm="l2")
tv1_fit = tv1.fit_transform(ALL)
print(tv1.get_feature_names())
# 得到每一个文本对应的TF-IDF向量
vsm_matrix = tv1_fit.toarray()
# vsm_matrix
textList = []
result_list = []
for i in range(LENGTH_TRAIN):
    textList.append(vsm_matrix[i])
textList = np.array(textList)
for i in range(LENGTH_TRAIN, LENGTH_ALL):
    result_list.append(vsm_matrix[i])
result_list = np.array(result_list)
# result_list

['00', '000', '0000', '000kms', '000ma', '006', '007', '008', '00am', '00pm', '01', '02', '0214', '03', '0394826922', '04', '05', '05750', '059', '06', '0616', '07', '08', '08525', '08for', '08i', '08kw', '09', '0f', '0so', '0w', '10', '100', '1000', '1001', '100c', '100ft', '100g', '100in', '100k', '100s', '100stars', '100x', '101', '1010', '1011', '1012', '1013', '1014', '1015', '1016', '101d', '101st', '102', '1020', '1022', '103', '10304', '104', '105', '105a', '106', '107', '108', '1080i', '1080p', '109', '10a', '10album', '10all', '10another', '10avoid', '10black', '10california', '10check', '10cool', '10d2', '10disc', '10ebony', '10final', '10good', '10got', '10great', '10have', '10heaven', '10holla', '10how', '10i', '10if', '10isn', '10it', '10joy', '10life', '10lyrics', '10m', '10min', '10mins', '10mm', '10my', '10ngiculela', '10no', '10one', '10only', '10ordinary', '10originality', '10oz', '10picture', '10production', '10ratha', '10run', '10shorty', '10skandulous', '10th', '1

### 四. 利用训练集数据训练多分类逻辑回归模型
使用sklearn中的LogisticRegression模块。
在这里，我们通过蒙特卡洛交叉验证来验证模型的正确性。

In [15]:
# 划分训练集与测试集，这里选取12.5%的数据作为测试集，剩余数据作为训练集。
# random_state数值是不同会让训练集与测试集不同，若写为None则每次都随机生成。
# accuracyTotal = 0
# LOOP_NUMBER = 3
# for loop in range(LOOP_NUMBER):
#     text_train, text_test, label_train, label_test = train_test_split(textList, labelList, test_size=0.125, random_state=None)
#     model = LogisticRegression(max_iter=1000)
#     model.fit(text_train, label_train)
#     accuracy = model.score(text_test, label_test)
#     accuracyTotal = accuracyTotal + accuracy
# print("模型准确率：", accuracyTotal / LOOP_NUMBER)

这说明模型准确率较高，可以投入使用。故我们将全部数据投入到模型中进行训练。

In [16]:
model = LogisticRegression(max_iter=1000)
model.fit(textList, labelList)

LogisticRegression(max_iter=1000)

### 五. 预测测试集结果
预测并保存最终的结果。

In [17]:
predictions = model.predict(result_list)
predictions

array([6, 1, 4, ..., 9, 6, 9])

In [18]:
with open("result1.txt", "w") as file:
    file.write('id, pred\n')
    for index, item in enumerate(predictions):
        file.write(f'{index}, {item}\n')

### 六. 稳定性检验
对训练集进行多次随机划分，对每次划分后的训练集进行逻辑回归，并将模型用于运行测试集。将得到的多个结果分别计算余弦相似度，从而观测减少一部分训练集的内容是否会对模型的预测结果产生巨大的影响。经过实验，发现随机地抽取87.5%的训练集进行模型的训练对于模型最后在测试集上的表现是没有过大区别的，这说明我们的模型具有较好的稳定性。

In [19]:
def cos_similar(vec1, vec2):
    return vec1.dot(vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

In [20]:
A = []
for i in range(3):
    text_train, text_test, label_train, label_test = train_test_split(textList, labelList, test_size=0.125, random_state=None)
    model = LogisticRegression(max_iter=1000, random_state=None)
    model.fit(text_train, label_train)
    A.append(model.predict(result_list))
cos1 = cos_similar(A[0], A[1])
cos2 = cos_similar(A[0], A[2])
cos3 = cos_similar(A[2], A[1])
print(cos1, cos2, cos3)

0.9954881779565797 0.9943370614583487 0.9950105135074548
